# Taiwan Social Change Survey (TSCS) Data Analysis
## 1992-2022 Time Series Analysis

This notebook imports and analyzes TSCS data from 1992 to 2022 (every 5 years) to identify common variables **by their meaning (labels)** across all survey years for time-series visualization.

**Important**: Variables may have different names (e.g., v_26 vs v_32) across years, but if they measure the same concept (e.g., "subjective wealth"), they are considered common variables.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import pyreadstat
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## Load TSCS Data Files

Loading all SPSS (.sav) files from the vi_data directory with metadata.

In [2]:
# Define data path and file mappings
data_path = Path('../vi_data')

# Map files to years
data_files = {
    1992: 'tscs921.sav',
    1997: 'tscs971_l.sav',
    2002: 'tscs021.sav',
    2007: 'tscs071.sav',
    2012: 'tscs121.sav',
    2017: 'tscs171.sav',
    2022: 'tscs221.sav'
}

# Dictionary to store dataframes and metadata
datasets = {}
metadata = {}

# Load each dataset
for year, filename in data_files.items():
    file_path = data_path / filename
    try:
        df, meta = pyreadstat.read_sav(str(file_path))
        datasets[year] = df
        metadata[year] = meta
        print(f"✓ Loaded {year}: {len(df)} records, {len(df.columns)} variables")
    except Exception as e:
        print(f"✗ Error loading {year}: {e}")

print(f"\nTotal datasets loaded: {len(datasets)}")

✓ Loaded 1992: 2377 records, 572 variables
✓ Loaded 1997: 2596 records, 495 variables
✓ Loaded 2002: 1992 records, 631 variables
✓ Loaded 2007: 2040 records, 739 variables
✓ Loaded 2012: 2134 records, 432 variables
✓ Loaded 2017: 1917 records, 990 variables
✓ Loaded 2022: 1739 records, 446 variables

Total datasets loaded: 7


## Extract Variable Labels for Each Year

Build mappings of variable names to their labels (meanings) for each survey year.

In [3]:
# Extract variable labels for each year
year_labels = {}  # {year: {var_name: label}}
label_to_vars = {} # {year: {label: var_name}}

for year in sorted(datasets.keys()):
    year_labels[year] = {}
    label_to_vars[year] = {}
    
    col_to_label = metadata[year].column_names_to_labels
    
    for var_name in datasets[year].columns:
        label = col_to_label.get(var_name, None)
        if label:
            # Clean up label (strip whitespace)
            label = label.strip()
            year_labels[year][var_name] = label
            label_to_vars[year][label] = var_name
    
    print(f"{year}: {len(year_labels[year])} variables with labels")

1992: 572 variables with labels
1997: 495 variables with labels
2002: 631 variables with labels
2007: 739 variables with labels
2012: 432 variables with labels
2017: 990 variables with labels
2022: 446 variables with labels


## Display Sample Labels from Each Year

In [4]:
# Show sample of labels from each year
for year in sorted(datasets.keys()):
    if year == 2017:
        print(f"\n{'='*70}")
        print(f"Year {year} - Sample Variable Labels:")
        print(f"{'='*70}")

        for i, (var, label) in enumerate(list(year_labels[year].items())):
            print(f"{var:15s} -> {label}")


Year 2017 - Sample Variable Labels:
id              -> id 受訪者編號
zip             -> zip (抽樣地區)郵遞區號
stratum2        -> stratum2 2005層別代碼
region          -> region 2005地理區代碼
stratum2014     -> stratum2014 2014層別代碼
region2014      -> region2014 2014地理區代碼
r_stratum2014   -> r_stratum201 2014地理區與層別代碼
psu             -> psu 第一抽出單位
ssu             -> ssu 第二抽出單位
wsel0           -> wsel0 不等機率加權權值(抽樣)
wsel            -> wsel 不等機率加權權值
wr_19_4         -> wr_19_4 建立在初始權值等於不等機率加權權值，且以19分層及教育程度4分類進行加權之多變項反覆加權權值
wr_19_5         -> wr_19_5 建立在初始權值等於不等機率加權權值，且以19分層及教育程度5分類進行加權之多變項反覆
wave            -> wave 波次
qtype           -> qtype 問卷別
year            -> year 調查年(西元年)
year_m          -> year_m 調查年(民國年)
sdt1            -> sdt 訪問開始日期
sdt2            -> sdt 訪問開始時間【24小時制】
a1              -> A1 性別
a2y             -> A2 請問您是什麼時候出生的?出生年
a2m             -> A2 請問您是什麼時候出生的?出生月
a2a             -> A2a 請問您大約幾歲?
a3city          -> A3 請問您在什麼地方出生的?
ka3city         -> A3 請問您在什麼地方出生的?(29)其他
a3zip           -> A3 請問您在什麼

In [5]:
# Invesgating Value Labels

year = 2017
target_var = "zip"

# --- 檢查變項是否存在 ---
if target_var in datasets[year].columns:
    
    # 3. 獲取選項對應 (Value Labels) - 即「答案選項是什麼」
    # meta.variable_value_labels 是一個字典 {變項名: {數值: 標籤}}
    val_labels = metadata[year].variable_value_labels.get(target_var, {})

    # --- 輸出結果 ---
    print(f"Variable: {target_var}")
    print("-" * 30)
    print("Value Labels:")
    
    if val_labels:
        for value, label in val_labels.items():
            # 數值通常是 float，這裡印出來時不一定要顯示小數點
            print(f"   {int(value)} = {label}")
    else:
        print("   (此變項沒有設定數值標籤，可能是連續變數或文字)")

    print("-" * 30)
    print(datasets[year][target_var].value_counts())

else:
    print(f"❌ 找不到變項 '{target_var}'。請檢查拼字或大小寫。")

Variable: zip
------------------------------
Value Labels:
   100 = 台北市中正區
   103 = 台北市大同區
   104 = 台北市中山區
   105 = 台北市松山區
   106 = 台北市大安區
   108 = 台北市萬華區
   110 = 台北市信義區
   111 = 台北市士林區
   112 = 台北市北投區
   114 = 台北市內湖區
   115 = 台北市南港區
   116 = 台北市文山區
   200 = 基隆市仁愛區
   201 = 基隆市信義區
   202 = 基隆市中正區
   203 = 基隆市中山區
   204 = 基隆市安樂區
   205 = 基隆市暖暖區
   206 = 基隆市七堵區
   207 = 新北市萬里區
   208 = 新北市金山區
   220 = 新北市板橋區
   221 = 新北市汐止區
   222 = 新北市深坑區
   223 = 新北市石碇區
   224 = 新北市瑞芳區
   226 = 新北市平溪區
   227 = 新北市雙溪區
   228 = 新北市貢寮區
   231 = 新北市新店區
   232 = 新北市坪林區
   233 = 新北市烏來區
   234 = 新北市永和區
   235 = 新北市中和區
   236 = 新北市土城區
   237 = 新北市三峽區
   238 = 新北市樹林區
   239 = 新北市鶯歌區
   241 = 新北市三重區
   242 = 新北市新莊區
   243 = 新北市泰山區
   244 = 新北市林口區
   247 = 新北市蘆洲區
   248 = 新北市五股區
   249 = 新北市八里區
   251 = 新北市淡水區
   252 = 新北市三芝區
   253 = 新北市石門區
   260 = 宜蘭縣宜蘭市
   261 = 宜蘭縣頭城鎮
   262 = 宜蘭縣礁溪鄉
   263 = 宜蘭縣壯圍鄉
   264 = 宜蘭縣員山鄉
   265 = 宜蘭縣羅東鎮
   266 = 宜蘭縣三星鄉
   267 = 宜蘭縣大同鄉
   268 = 宜蘭縣五結鄉
   269 = 宜蘭縣冬山鄉
   270 = 宜蘭縣蘇

## Find Common Labels Across All Years

Identify variable labels (meanings) that appear in ALL survey years, regardless of variable name.

In [5]:
# Get all labels for each year
label_sets = {year: set(label_to_vars[year].keys()) for year in sorted(datasets.keys())}

# Find labels that exist in all years
common_labels = set.intersection(*label_sets.values())

print(f"Total labels appearing in ALL years: {len(common_labels)}")
print(f"\nFirst 20 common labels:")
print("="*70)
for i, label in enumerate(sorted(common_labels)[:20]):
    print(f"{i+1:3d}. {label}")

Total labels appearing in ALL years: 1

First 20 common labels:
  1. qtype 問卷別


## Create Variable Mapping Across Years

For each common label, show which variable name represents it in each year.

In [ ]:
# Create mapping: label -> {year: variable_name}
label_mappings = {}

for label in common_labels:
    label_mappings[label] = {}
    for year in sorted(datasets.keys()):
        if label in label_to_vars[year]:
            label_mappings[label][year] = label_to_vars[year][label]

# Display sample mappings
print("Sample Variable Mappings (first 10 common concepts):")
print("="*80)
for i, (label, year_vars) in enumerate(sorted(label_mappings.items())[:10]):
    print(f"\n{i+1}. {label}")
    for year in sorted(year_vars.keys()):
        print(f"   {year}: {year_vars[year]}")

## Check Data Completeness for Common Labels

Assess missing data rates for variables representing each common concept.

In [ ]:
# Calculate missing data for each common label across years
label_completeness = []

for label in sorted(common_labels):
    row = {'Label': label}
    missing_rates = []
    
    for year in sorted(datasets.keys()):
        var_name = label_mappings[label][year]
        missing_pct = (datasets[year][var_name].isna().sum() / len(datasets[year])) * 100
        row[str(year)] = f"{missing_pct:.1f}%"
        missing_rates.append(missing_pct)
    
    row['Max_Missing'] = max(missing_rates)
    label_completeness.append(row)

completeness_df = pd.DataFrame(label_completeness)

print("\nMissing Data Percentage by Year (first 20 variables):")
print("="*100)
display_cols = ['Label'] + [str(y) for y in sorted(datasets.keys())] + ['Max_Missing']
print(completeness_df[display_cols].head(20).to_string(index=False))

## Identify High-Quality Variables for Visualization

Filter concepts with < 10% missing data across all years and are suitable for time-series visualization.

In [ ]:
# Find high-quality variables (< 10% missing)
high_quality_labels = []

for label in common_labels:
    max_missing = 0
    for year in sorted(datasets.keys()):
        var_name = label_mappings[label][year]
        missing_pct = (datasets[year][var_name].isna().sum() / len(datasets[year])) * 100
        max_missing = max(max_missing, missing_pct)
    
    if max_missing < 10:
        high_quality_labels.append(label)

print(f"High-quality concepts (< 10% missing in all years): {len(high_quality_labels)}")
print("\nRecommended concepts for time-series visualization:")
print("="*70)
for i, label in enumerate(sorted(high_quality_labels)[:30], 1):
    # Show variable names across years
    vars_str = " -> ".join([label_mappings[label][year] for year in sorted(datasets.keys())])
    print(f"{i:3d}. {label}")
    print(f"     Variables: {vars_str}")

## Analyze Variable Types and Value Ranges

Understand what types of data these common concepts contain.

In [ ]:
# Analyze high-quality variables
variable_analysis = []

for label in sorted(high_quality_labels)[:20]:  # First 20 for detailed analysis
    # Get variable from first year as reference
    first_year = sorted(datasets.keys())[0]
    var_name = label_mappings[label][first_year]
    
    # Get basic stats
    data_sample = datasets[first_year][var_name].dropna()
    
    analysis = {
        'Label': label,
        'Type': str(data_sample.dtype),
        'Unique_Values': data_sample.nunique(),
        'Sample_Values': str(sorted(data_sample.unique())[:5])
    }
    
    variable_analysis.append(analysis)

analysis_df = pd.DataFrame(variable_analysis)
print("\nVariable Type Analysis (first 20 high-quality concepts):")
print("="*100)
print(analysis_df.to_string(index=False))

## Sample Analysis: Time-Series for a Specific Concept

Demonstrate how a concept's values change over the 30-year period.

In [ ]:
# Pick a high-quality label for demonstration
if high_quality_labels:
    sample_label = sorted(high_quality_labels)[0]
    print(f"Time-Series Analysis for: {sample_label}")
    print("="*70)
    
    for year in sorted(datasets.keys()):
        var_name = label_mappings[sample_label][year]
        print(f"\n{year} (variable: {var_name}):")
        value_counts = datasets[year][var_name].value_counts().head(10)
        print(value_counts)
        
        # Check if there are value labels
        if var_name in metadata[year].variable_value_labels:
            value_labels = metadata[year].variable_value_labels[var_name]
            print("\nValue Labels:")
            for val, label in list(value_labels.items())[:10]:
                print(f"  {val}: {label}")
else:
    print("No high-quality variables found for sample analysis")

## Find Labels Present in Most Years (Not Necessarily All)

Some important concepts may exist in 5-6 years but not all 7.

In [ ]:
# Count how many years each label appears in
label_year_counts = defaultdict(list)

for year in sorted(datasets.keys()):
    for label in label_to_vars[year].keys():
        label_year_counts[label].append(year)

# Find labels in at least 5 years
frequent_labels = {label: years for label, years in label_year_counts.items() 
                   if len(years) >= 5}

print(f"Labels appearing in at least 5 out of 7 years: {len(frequent_labels)}")
print("\nSample of labels in 5-6 years (first 20):")
print("="*70)

count = 0
for label, years in sorted(frequent_labels.items()):
    if len(years) < 7 and count < 20:  # Show only those NOT in all years
        count += 1
        print(f"{count}. {label}")
        print(f"   Years: {years}")

## Export Comprehensive Summary for Visualization

In [ ]:
import json

# Create comprehensive summary
summary = {
    'survey_years': list(sorted(datasets.keys())),
    'total_common_labels': len(common_labels),
    'high_quality_labels': len(high_quality_labels),
    'label_mappings': {},  # {label: {year: var_name}}
    'high_quality_concepts': [],
    'frequent_labels_5plus_years': len(frequent_labels)
}

# Add complete mappings for high-quality labels
for label in high_quality_labels:
    summary['label_mappings'][label] = label_mappings[label]
    
    # Add detailed info
    first_year = sorted(datasets.keys())[0]
    var_name = label_mappings[label][first_year]
    
    concept_info = {
        'label': label,
        'variables_by_year': label_mappings[label],
        'data_type': str(datasets[first_year][var_name].dtype),
        'unique_values': int(datasets[first_year][var_name].nunique())
    }
    
    summary['high_quality_concepts'].append(concept_info)

# Save summary
with open('tscs_variable_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

print("✓ Summary exported to: tscs_variable_summary.json")
print(f"\nSummary Statistics:")
print(f"  - Survey years: {summary['survey_years']}")
print(f"  - Common labels (all 7 years): {summary['total_common_labels']}")
print(f"  - High-quality labels (< 10% missing): {summary['high_quality_labels']}")
print(f"  - Labels in 5+ years: {summary['frequent_labels_5plus_years']}")

## Create Detailed Report of Top Concepts

In [ ]:
# Create a detailed report for easy review
print("="*80)
print("TOP 30 CONCEPTS FOR TIME-SERIES VISUALIZATION")
print("="*80)

for i, label in enumerate(sorted(high_quality_labels)[:30], 1):
    print(f"\n{i}. {label}")
    print("-" * 70)
    
    # Show variable mapping
    print("Variable names by year:")
    for year in sorted(datasets.keys()):
        var_name = label_mappings[label][year]
        n_valid = datasets[year][var_name].notna().sum()
        n_total = len(datasets[year])
        print(f"  {year}: {var_name:15s} ({n_valid}/{n_total} valid responses)")
    
    # Show sample values from first year
    first_year = sorted(datasets.keys())[0]
    var_name = label_mappings[label][first_year]
    unique_vals = datasets[first_year][var_name].nunique()
    print(f"\nUnique values: {unique_vals}")
    
    # Show value distribution from first year
    print("\nValue distribution (first year):")
    print(datasets[first_year][var_name].value_counts().head())

## Conclusion

This analysis identified variables **by their semantic meaning (labels)** that exist across all TSCS survey years (1992-2022). Even though variable names may differ across years (e.g., v_26 in 1992 vs v_32 in 1997), if they measure the same concept (e.g., "subjective wealth"), they are tracked together.

**Key Findings**:
- Identified common concepts that can be tracked over 30 years
- Created mappings showing which variable represents each concept in each year
- Filtered for data quality to ensure reliable visualization
- Prepared structured data for d3.js time-series visualization

**Next Steps**:
1. Review the top concepts and select those most interesting for visualization
2. Integrate selected concepts with the Taiwan geographical map
3. Create interactive time-series animations showing change over 30 years